### Set standard imports

In [88]:
import sys,os
import matplotlib
import matplotlib.pyplot as plt
import lmfit
import numpy as np
import pandas
#if this fails change qt to tk
%matplotlib qt 
import KiMoPack.plot_func as pf
from importlib import reload
reload(pf)

Plot_func version 6.11.3
was imported from path:
 H:\Programs\Anaconda\Lib\site-packages\KiMoPack
The current working folder is:
 H:\Dropbox\coding\github\KiMoPack\Workflow_tools


<module 'KiMoPack.plot_func' from 'H:\\Programs\\Anaconda\\lib\\site-packages\\KiMoPack\\plot_func.py'>

In [118]:
import scipy.constants as const
import import_library
reload(import_library)
from import_library import Ivan_horse
ta=pf.TA('recent',conversion_function=Ivan_horse)

In [137]:
def horse(ds,wavelength=485e-9,fit=False,cmap=None,ax=None):
    if cmap is None:
        colors=pf.colm(range(len(ta.ds.columns)))
    else:
        colors=pf.colm(range(len(ta.ds.columns)),cmap=cmap)
    plot_dicten={}
    for col in ta.ds.columns:
        per_photon=const.h*const.c/(485e-9)
        index=col*ta.ds.index.values*per_photon
        plot_dicten['%.2g'%col]=pandas.Series(ta.ds.loc[:,col].values,index=index)
    if ax is None:
        fig,ax=plt.subplots()
    for i,key in enumerate(plot_dicten.keys()):
        if fit:
            plot_dicten[key].plot(ax=ax,style='-',color=colors[i])
        else:
            plot_dicten[key].plot(ax=ax,style='o',label='Fluence in %s ph/$cm^2 s$'%key,color=colors[i])
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xlabel('Power density in $cm^2$')
    ax.set_ylabel('PLQY')
    if not fit:
        ax.legend()
    return ax
plt.close('all')
fig,ax=plt.subplots()
ax=horse(ta.ds,ax=ax)
horse(ta.ds,ax=ax,fit=True)
fig.tight_layout()

In [ ]:
def manual_consecutive(times,pardf):								
	'''we have 1MLCT,3MLCT,3MC state with 
	consecutive decays, hint this function could have been created with the linear exp model
	'''
	c=np.zeros((len(times),3),dtype='float') 						#creation of matrix that will hold the concentrations
	g=gauss(times,sigma=pardf['resolution']/FWHM,mu=pardf['t0']) 	#creating the gaussian pulse that will "excite" our sample
	sub_steps=10 													#defining how many extra steps will be taken between the main time_points
	for i in range(1,len(times)):									#iterate over all timepoints
		dc=np.zeros((3,1),dtype='float')							#the initial change for each concentration, the "3" is representative of how many changes there will be
		dt=(times[i]-times[i-1])/(sub_steps)						# as we are taking smaller steps the time intervals need to be adapted
		c_temp=c[i-1,:]												#temporary matrix holding the changes (needed as we have sub steps and need to check for zero in the end)
		for j in range(int(sub_steps)):
			dc[0]=-pardf['k0']*dt*c_temp[0]+g[i]*dt					#excite a small fraction with g[i] and decay with 'k0'
			dc[1]=pardf['k0']*dt*c_temp[0]-pardf['k1']*dt*c_temp[1]	#form with "k0" and decay with "k1"
			dc[2]=pardf['k1']*dt*c_temp[1]-pardf['k2']*dt*c_temp[2]	#form with "k1" and decay with "k2"
			for b in range(c.shape[1]):
				c_temp[b] =np.nanmax([(c_temp[b]+float(dc[b])),0.])		#check that nothing will be below 0 (concentrations)
		c[i,:] =c_temp												#store the temporary concentrations into the main matrix
	c=pandas.DataFrame(c,index=times)								#write back the right indexes
	c.index.name='time'												#and give it a name
	c.columns=['1MLCT','3MLCT','3MC']								#this is optional but very useful. The species get names that represent some particular states
	if 'background' in list(pardf.index.values):					#optional but usefull, allow the keyword "background" to be used to fit the background in the global analysis
		c['background']=1											#background always there (flat)
	return c					

### Perform fast fitting using independent exponential decay (first order)

In [ ]:
plt.close('all')                                             # Close old spectra
ta.mod='exponential'                                         # Choose the model 'exponential', 'consecutive' and 'fast_consecutive'are build in
par=lmfit.Parameters()                                       # create empty parameter object

par.add('k0',value=1/0.1,vary=True)                                # Add second rate component
par.add('k1',value=1/2.5,vary=True)                                # Add one rate component
par.add('k2',value=1/40,vary=True)                                 # Add second rate component
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust (for all)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (for all)
#par.add('infinite')                                                # Keyword for an non decaying component
#par.add('background')                                             # Keyword to fit the Background Disable the background substraction in the loading cell for best effect
#ta.timelimits=[0.2,5000]                                      # select time window in which to fit
ta.ignore_time_region=[-0.1,0.5]                              # ignore the region over t0 for the fit
ta.log_fit=False                                               # fit in linear or log scale
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global(fit_chirp=False)                                 # trigger fitting

ta.error_matrix_amplification=10;                              # Choose how much stronger should error be plotted       
ta.log_scale=False                                             # 2D plots in linear or log scale
ta.Plot_fit_output(4)                                 # plot the fit output